In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
import torch
import os
from IPython.display import Image, clear_output

print (f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available()else 'CPU'})")

Setup complete. Using torch 2.9.0+cu128 (NVIDIA GeForce RTX 5070 Ti)


In [3]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="3Irr39DHWqAkEhIqAHU9")
project = rf.workspace("road-damage-detection-n2xkq").project("crack-and-pothole-bftyl")
version = project.version(1)
dataset = version.download("yolov12")

loading Roboflow workspace...
loading Roboflow project...


In [4]:
from ultralytics import YOLO

# Load a COCO-pretrained YOLO12n model
model = YOLO("yolo12n.pt")


In [5]:
# Train
results = model.train(
    data="crack-and-pothole-1/data.yaml",
    epochs=100,
    patience=10,
    batch=32,
    imgsz=640,
    mosaic=0.1,     # biar konteks jalan terjaga
    hsv_h=0.015,    
    hsv_s=0.5,
    hsv_v=0.3,
    lr0=0.001,      # learning rate kecil biar halus
)

Ultralytics 8.3.225 🚀 Python-3.12.11 torch-2.9.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5070 Ti, 15851MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=crack-and-pothole-1/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.5, hsv_v=0.3, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo12n.pt, momentum=0.937, mosaic=0.1, multi_scale=False, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10, perspective=0.

In [6]:
# Validasi
result_val = model.val()

Ultralytics 8.3.225 🚀 Python-3.12.11 torch-2.9.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5070 Ti, 15851MiB)
YOLOv12n summary (fused): 159 layers, 2,557,118 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3807.6±1510.8 MB/s, size: 72.4 KB)
val: Scanning /workspace/crack-and-pothole-1/valid/labels.cache... 731 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 731/731 3.7Mit/s 0.0s0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 46/46 19.8it/s 2.3s0.1s
                   all        731       1767      0.882      0.783       0.83      0.556
                 crack        370        884       0.91       0.91      0.926       0.64
               pothole        361        883      0.854      0.656      0.735      0.473
Speed: 0.4ms preprocess, 1.5ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to /workspace/runs/detect/val2


In [13]:
# Export ke ONNX
success = model.export(format='onnx', device='cpu')

Ultralytics 8.3.225 🚀 Python-3.12.11 torch-2.9.0+cu128 CPU (Intel Core i5-14600KF)

PyTorch: starting from '/workspace/runs/detect/train3/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 6, 8400) (5.3 MB)

ONNX: starting export with onnx 1.19.1 opset 22...
ONNX: slimming with onnxslim 0.1.73...
ONNX: export success ✅ 0.7s, saved as '/workspace/runs/detect/train3/weights/best.onnx' (10.1 MB)

Export complete (0.9s)
Results saved to /workspace/runs/detect/train3/weights
Predict:         yolo predict task=detect model=/workspace/runs/detect/train3/weights/best.onnx imgsz=640  
Validate:        yolo val task=detect model=/workspace/runs/detect/train3/weights/best.onnx imgsz=640 data=crack-and-pothole-1/data.yaml  
Visualize:       https://netron.app


In [8]:
!ls /content/yolov5/runs/train/

ls: cannot access '/content/yolov5/runs/train/': No such file or directory


In [9]:
!cp -r /content/yolov5/runs/train/exp /content/drive/MyDrive/

cp: cannot stat '/content/yolov5/runs/train/exp': No such file or directory


In [10]:
from IPython.display import Image
Image(filename='runs/train/exp/results.png')

FileNotFoundError: [Errno 2] No such file or directory: 'runs/train/exp/results.png'